# Images in Weave

- upload or generate images
- add them to lists or Tables
- Facet across these
- dynamic & interactive Weave Ops via Facet

In [ ]:
import typing
import itertools
from PIL import Image
import weave
from weave.show import show

## Show an image

In [ ]:
show(Image.linear_gradient("L"))

## Images in Lists

In [ ]:
ims = [Image.linear_gradient("L").rotate(i * 4) for i in range(100)]
show(ims)

## Images in Tables

In [ ]:
ims = []
base_im = Image.linear_gradient('L')
for rotate in range(10):
    for shear in range(10):
        ims.append({
            'rotate': rotate,
            'shear': shear,
            'image': base_im
                .rotate(rotate * 4)
                .transform((256, 256), Image.AFFINE, (1, shear / 10, 0, 0, 1, 0), Image.BICUBIC)
        })
ims = weave.save(ims)
show(ims)

## Facet is fun!

In [ ]:
facet = weave.panels.Facet(ims,
                           x=lambda row: row['rotate'],
                           y=lambda row: row['shear'],
                           select=lambda row: row[0]['image'])
show(facet, height=600)

## Create some ops to make it dynamic and interactive

In [ ]:
# weave-test-skip: in notebook declared ops can't run on server yet

class ParamType(typing.TypedDict):
    rotate: int
    shear: int
        
@weave.op(render_info={'type': 'function'})
def make_params(n_rotate: int, n_shear: int) -> list[ParamType]:
    params = []
    for rotate, shear in itertools.product(range(n_rotate), range(n_shear)):
        params.append({
            'rotate': rotate,
            'shear': shear
        })
    return params

@weave.op(render_info={'type': 'function'})
def make_image(rotate: int, shear: int) -> Image.Image:
    return (Image.linear_gradient('L')
        .rotate(rotate * 4)
        .transform((256, 256), Image.AFFINE, (1, shear / 10, 0, 0, 1, 0), Image.BICUBIC))

facet = weave.panels.Facet(make_params(10, 5),
                           x=lambda row: row['rotate'],
                           y=lambda row: row['shear'],
                           select=lambda row: make_image(row[0]['rotate'], row[0]['shear']))
show(facet, height=600)